In [1]:
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import torch

# get our predefined functions (for data processing, etc.)
import QCircNet.utils as ut

# get our predefined quantum circuits and neural networks
import QCircNet.circuits.CustomCircuits as cn
import QCircNet.QuantumNN as qnn


EXPERIMENT_ID = 69

DATA_PATH = "data/"

N_QUBITS = 4
FEAT_PER_QUBIT = 4

SUBSET = 1000

EPOCHS = 10
WEIGHT_PATH = f"results/weights/"
MODEL_PATH = f"results/models/"
RESULTS_PATH = f"results/model_10_results_expID_{EXPERIMENT_ID}.csv"
RESULTS_DF = pd.DataFrame()

# for printing colored text output during training
RED_TEXT = "\033[91m"
GREEN_TEXT = "\033[92m"
RESET_COLOR = "\033[0m" 


In [2]:
# load the data
X_train, y_train, X_test, y_test  = ut.load_MNIST_data(DATA_PATH, subset=SUBSET)

# check shapes
print("train:\t", X_train.shape, y_train.shape)
print("test:\t", X_test.shape, y_test.shape)

train:	 torch.Size([1000, 16]) torch.Size([1000, 10])
test:	 torch.Size([1000, 16]) torch.Size([1000, 10])


In [3]:
cn.get_custom_circuits()

[QCircNet.circuits.CustomCircuits.Anusha2024Circuit,
 QCircNet.circuits.CustomCircuits.DoubleEntanglementVLCircuit,
 QCircNet.circuits.CustomCircuits.Ranga2024Circuit,
 QCircNet.circuits.CustomCircuits.Senokosov2024Circuit]

In [4]:
circuits = [
    # cn.Ranga2024Circuit,
    cn.Senokosov2024Circuit,
    # cn.Anusha2024Circuit,
    # cn.DoubleEntanglementVLCircuit,
]

runs = list(range(1, 2)) # runs are also used as seeds (reproducibility)
learning_rates = [0.01, 0.001]

In [ ]:
### RUN EXPERIMENT
id = 0
for seed, circuit, lr in itertools.product(runs, circuits, learning_rates):
    print(f"""{RED_TEXT}RUN {id+1}:{RESET_COLOR}\n {circuit.__name__} \n lr: {lr}""")

    # init the model
    model = qnn.QuantumClassifierMNIST(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=seed)

    path_identifier = f"{circuit.__name__}_expID_{EXPERIMENT_ID}_dfID_{id}"

    # train the model (predifined optimizer, etc.)
    trained_model, loss_history, avg_epoch_time = ut.train_model(
        model, X_train, y_train,
        epochs=EPOCHS, batch_size=32, lr=lr, binary=True,
        weight_path=WEIGHT_PATH+path_identifier
    )
    
    # save trained model
    model_path = MODEL_PATH+path_identifier+".pth"
    torch.save(trained_model, model_path)

    # evaluate
    cross_entropy, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)

    results_dict = {
        "id": id,
        "f1_score": f1, "precision": precision, "recall": recall, "accuracy": accuracy, "cross_entropy": cross_entropy,
        "seed": seed,
        "circuit": circuit.__name__,
        "learning_rate": lr,
        "loss_history": loss_history,
        "avg_epoch_time_seconds": avg_epoch_time,
        "network_str": str(model),
        "model_path": model_path,
        "weight_path": WEIGHT_PATH,
        "epochs": EPOCHS,
        "n_qubits": N_QUBITS,
        "features_per_qubit": FEAT_PER_QUBIT,
        # "scaler" : str(SCALER).replace("()", "")
    }
    print(f"\n{GREEN_TEXT}RESULTS:\n F1-Score:{f1}, Precision:{precision}, Recall:{recall}, Accuracy:{accuracy}, Loss:{cross_entropy} {RESET_COLOR} \n\n")
    
    # append the results_dict to the RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([results_dict])], ignore_index=True)
    RESULTS_DF.to_csv(path_or_buf=RESULTS_PATH) # save for each run (in case of a crash)

    id += 1

RUN 1:
 Senokosov2024Circuit 
 lr: 0.01


Training progress:   0%|          | 0/31 [00:00<?, ?batch/s]

None
tensor([[-1.0427,  2.7687, -6.2817, -2.4840, -4.4390, -5.1228, -3.9426, -1.9407,
         -1.2973],
        [ 0.4878, -1.0154,  2.3275, -3.7140,  4.7516, -5.9390,  2.1422, -1.0392,
          0.7375],
        [-4.5190, -3.7938,  3.7793,  5.8843, -2.3446,  2.4168,  4.7298,  4.9588,
         -5.2145],
        [-5.7924, -4.1490,  4.7519, -5.0473, -0.9914,  5.7540,  0.4168,  2.4112,
         -2.3183]])


Training progress:   3%|▎         | 1/31 [00:01<00:33,  1.11s/batch]

None
None
tensor([[-1.0327,  2.7787, -6.2917, -2.4740, -4.4490, -5.1228, -3.9426, -1.9407,
         -1.2973],
        [ 0.4778, -1.0254,  2.3375, -3.7240,  4.7616, -5.9390,  2.1422, -1.0392,
          0.7375],
        [-4.5290, -3.8038,  3.7693,  5.8943, -2.3546,  2.4168,  4.7298,  4.9588,
         -5.2145],
        [-5.7824, -4.1390,  4.7619, -5.0573, -1.0014,  5.7540,  0.4168,  2.4112,
         -2.3183]])


...

None
None
tensor([[-0.9424,  2.9126, -6.4117, -2.3468, -4.5795, -5.1228, -3.9426, -1.9407,
         -1.2973],
        [ 0.4063, -1.1690,  2.4087, -3.8628,  4.8365, -5.9390,  2.1422, -1.0392,
          0.7375],
        [-4.6655, -3.9307,  3.6550,  6.0125, -2.4820,  2.4168,  4.7298,  4.9588,
         -5.2145],
        [-5.6437, -4.0075,  4.9020, -5.1995, -1.0777,  5.7540,  0.4168,  2.4112,
         -2.3183]])


Training progress:  52%|█████▏    | 16/31 [00:18<00:17,  1.15s/batch]

None
None
tensor([[-0.9419,  2.9212, -6.4177, -2.3396, -4.5879, -5.1228, -3.9426, -1.9407,
         -1.2973],
        [ 0.4107, -1.1795,  2.4050, -3.8723,  4.8342, -5.9390,  2.1422, -1.0392,
          0.7375],
        [-4.6744, -3.9378,  3.6503,  6.0179, -2.4894,  2.4168,  4.7298,  4.9588,
         -5.2145],
        [-5.6346, -3.9996,  4.9118, -5.2097, -1.0762,  5.7540,  0.4168,  2.4112,
         -2.3183]])


Training progress:  55%|█████▍    | 17/31 [00:20<00:16,  1.20s/batch]

None
None
tensor([[-0.9420,  2.9296, -6.4232, -2.3328, -4.5962, -5.1228, -3.9426, -1.9407,
         -1.2973],
        [ 0.4161, -1.1899,  2.4002, -3.8817,  4.8311, -5.9390,  2.1422, -1.0392,
          0.7375],
        [-4.6832, -3.9445,  3.6462,  6.0228, -2.4965,  2.4168,  4.7298,  4.9588,
         -5.2145],
        [-5.6257, -3.9921,  4.9214, -5.2199, -1.0742,  5.7540,  0.4168,  2.4112,
         -2.3183]])


In [ ]:
RESULTS_DF

,id,f1_score,precision,recall,accuracy,cross_entropy,seed,circuit,learning_rate,loss_history,avg_epoch_time_seconds,network_str,model_path,weight_path,epochs,n_qubits,features_per_qubit
0,0,0.113208,1.0,0.06,0.06,2.203273,1,Senokosov2024Circuit,0.010,"[0.2503029704093933, 0.2477768361568451]",0.564571,QuantumClassifierMNIST(\n (quantum_circuit_nn...,results/models/Senokosov2024Circuit_expID_69_d...,results/weights/,2,4,4
1,1,0.000000,0.0,0.00,0.00,2.210763,1,Senokosov2024Circuit,0.001,"[0.2503029704093933, 0.25011005997657776]",0.537497,QuantumClassifierMNIST(\n (quantum_circuit_nn...,results/models/Senokosov2024Circuit_expID_69_d...,results/weights/,2,4,4


In [ ]:
# circuit = qc.QuantumCircuit

# # initialize the model
# model = qnn.QuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=1)

# # train the model (predifined optimizer, etc.)
# trained_model, loss_history, val_loss_history = ut.train_model(
#     model, X_train, y_train, epochs=EPOCHS, batch_size=64, lr=0.01
# ) #X_val, y_val,


# # Evaluate the model
# bce, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)
# print(f"Binary Cross-Entropy on test data: {bce:.4f}")
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")

In [ ]:
# # To load later
# loaded_model = torch.load("simple_model.pth")